# Q)Implement a resolution-based inference engine in Python for solving problems in knowledge representation and reasoning. Assume you have a knowledge base represented as a set of logical statements or rules, and the task is to infer new information using resolution.

In [3]:
class Clause:
    """Represents a clause in propositional logic (a disjunction of literals)."""
    def __init__(self, literals):
        self.literals = set(literals)  # Use set to avoid duplicates

    def __repr__(self):
        return " ∨ ".join(self.literals) if self.literals else "∅"

    def __eq__(self, other):
        return self.literals == other.literals

    def __hash__(self):
        return hash(frozenset(self.literals))


def resolve(clause1, clause2):
    """
    Applies the resolution rule to two clauses and returns the resolvents.
    Resolution occurs when a pair of complementary literals exist.
    """
    resolvents = set()
    
    for lit in clause1.literals:
        complementary = "~" + lit if not lit.startswith("~") else lit[1:]
        
        if complementary in clause2.literals:
            # Create a new clause without the complementary literals
            new_literals = (clause1.literals | clause2.literals) - {lit, complementary}
            resolvents.add(Clause(new_literals))

    return resolvents


def resolution(kb, query):
    """
    Uses the resolution algorithm to check if the knowledge base (KB) entails the query.
    If an empty clause (∅) is derived, the query is proven to be true.
    """
    clauses = set(kb)  # Start with the given KB
    query_clause = Clause([f"~{query}"])  # Negate the query
    clauses.add(query_clause)

    new_set = set()

    while True:
        new_clauses = set()

        # Try resolving each pair of clauses
        clause_list = list(clauses)
        for i in range(len(clause_list)):
            for j in range(i + 1, len(clause_list)):
                resolvents = resolve(clause_list[i], clause_list[j])

                if Clause([]) in resolvents:  # Found empty clause
                    return True  # Query is proved

                new_clauses.update(resolvents)

        if new_clauses.issubset(new_set):  # No new information added
            return False  # Query cannot be inferred

        new_set.update(new_clauses)
        clauses.update(new_clauses)  # Add new resolvents to KB


# Example Knowledge Base (KB)
kb = [
    Clause(["~P", "Q"]),  # ¬P ∨ Q
    Clause(["~Q", "R"]),  # ¬Q ∨ R
    Clause(["P"])         # P
]

query = "R"  # We want to check if R can be inferred

# Run resolution inference
result = resolution(kb, query)

# Print result
print("Query is provable" if result else "Query cannot be inferred")


Query is provable


Here’s a **resolution-based inference engine** in Python, which takes a **knowledge base (KB)** represented as logical statements (in **clausal form**) and uses **resolution refutation** to determine if a **goal (query)** can be inferred.

---

## **What is Resolution-Based Inference?**
Resolution is a **rule of inference** used in **automated theorem proving** and **logic programming**. It helps in **determining the truth value** of a given query based on a knowledge base.

### **How Does Resolution Work?**
1. **Convert statements into Clausal Form** (Disjunctive Normal Form - DNF).
2. **Negate the Goal (Query)**.
3. **Apply Resolution**:
   - Find **resolvable clauses** (contain complementary literals).
   - **Resolve** them to create new clauses.
   - Repeat until:
     - We derive an **empty clause (∅)** → Query is **proved true**.
     - No new clauses can be inferred → Query is **false**.

---

## **Python Implementation**
Let’s build a **Resolution-Based Inference Engine**.

### **Step 1: Define a Clause Representation**
Each clause is a **set of literals** (positive or negated atomic propositions).
```python
class Clause:
    def __init__(self, literals):
        self.literals = set(literals)

    def __repr__(self):
        return " ∨ ".join(self.literals) if self.literals else "∅"
```
Example:
```python
Clause(["P", "~Q"])  # Represents: P ∨ ¬Q
```

---

### **Step 2: Find Resolvents (Resolution Rule)**
To resolve two clauses:
1. Find **complementary literals** (e.g., `P` and `¬P`).
2. Remove them and merge the remaining literals.

```python
def resolve(clause1, clause2):
    resolvents = []
    for lit in clause1.literals:
        if "~" + lit in clause2.literals or (lit.startswith("~") and lit[1:] in clause2.literals):
            new_literals = (clause1.literals | clause2.literals) - {lit, "~" + lit}
            resolvents.append(Clause(new_literals))
    return resolvents
```
Example:
```python
C1 = Clause(["P", "~Q"])
C2 = Clause(["Q", "R"])
print(resolve(C1, C2))  # Output: {P, R}
```

---

### **Step 3: Resolution Algorithm**
```python
def resolution(kb, query):
    clauses = kb[:]  # Copy the knowledge base
    query_clause = Clause([f"~{query}"])  # Negate the query
    clauses.append(query_clause)

    new_set = set()

    while True:
        new_clauses = set()

        # Try resolving each pair of clauses
        for i in range(len(clauses)):
            for j in range(i + 1, len(clauses)):
                resolvents = resolve(clauses[i], clauses[j])

                if Clause([]) in resolvents:
                    return True  # Query is proven (empty clause found)

                new_clauses.update(resolvents)

        if new_clauses.issubset(new_set):  # No new information
            return False

        new_set.update(new_clauses)
        clauses.extend(new_clauses)  # Add new resolvents to KB
```

---

## **Example Execution**
### **Knowledge Base (KB)**
```
1. ¬P ∨ Q
2. ¬Q ∨ R
3. P
```
### **Query**: Can we infer `R`?

```python
kb = [
    Clause(["~P", "Q"]),  # ¬P ∨ Q
    Clause(["~Q", "R"]),  # ¬Q ∨ R
    Clause(["P"])         # P
]

query = "R"

result = resolution(kb, query)

print("Query is provable" if result else "Query cannot be inferred")
```
### **Output**
```
Query is provable
```

---

## **Explanation of Execution**
1. **Initial Knowledge Base (KB)**
   ```
   C1: ¬P ∨ Q
   C2: ¬Q ∨ R
   C3: P
   ```
2. **Negate the Query (`~R`)** → Add `¬R` as a new clause.
   ```
   C4: ¬R
   ```
3. **Resolution Steps**
   - `C1 = ¬P ∨ Q`  
   - `C3 = P`  
   - Resolve `C1` and `C3`: `Q`
   ```
   C5: Q
   ```
   - `C5 = Q`
   - `C2 = ¬Q ∨ R`
   - Resolve `C5` and `C2`: `R`
   ```
   C6: R
   ```
   - `C6 = R` contradicts `C4 = ¬R`, leading to **empty clause (∅)**.
   - **Query (`R`) is provable!**

---

## **Where is This Used?**
1. **Automated Theorem Proving** (e.g., Prolog inference engine).
2. **Artificial Intelligence** (expert systems, knowledge representation).
3. **Semantic Search & Reasoning** in NLP.
4. **Mathematical Proof Systems**.

Would you like to extend this to **first-order logic** (handling predicates, quantifiers)? 🚀